<a href="https://colab.research.google.com/github/NivedhaBalakrishnan/Quaker-Jack/blob/main/Jack_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U openai-whisper -q  # https://github.com/openai/whisper
!pip install gradio -q             # https://gradio.app/quickstart/
!pip install openai -q             # https://github.com/openai/openai-python
!pip install elevenlabs -q                # https://github.com/coqui-ai/TTS
!pip install langchain
!pip install numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.3/794.3 kB 8.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.3/298.3 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [ ]:
import whisper
import gradio as gr
import openai
from elevenlabs import generate, save, set_api_key

import warnings
warnings.filterwarnings('ignore')

## Text to Speech (TTS) part

In [ ]:
from elevenlabs import set_api_key
set_api_key("")



## Whisper part

In [ ]:
# model can be passed as dropdown in gradio UI
model = whisper.load_model("tiny.en")
model.device

100%|█████████████████████████████████████| 72.1M/72.1M [00:20<00:00, 3.70MiB/s]


device(type='cuda', index=0)

## ChatGPT part

In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [ ]:

openai_api_key = ''
llm_model = "gpt-3.5-turbo"

In [ ]:
starbucks_template = """
You are a funny, helpful barista bot works at Starbucks. \
Your name is Jack. You love Starbucks coffee.\
You convince people to go to Starbucks. \
You have a natural conversation with people. \
You tell everyone that you love Starbuck's caramel Macchiato. \
You are a bot who works as a Starbucks barista. \
You love your customers and you take care of them. \
You tell a joke about Starbucks and ask customers what they want to order. \
Strictly keep your response below 300 characters. \

Here is a conversation:
{input}"""

apple_template = """Your name is Jack. \
You love apple products. \
You are excited to talk about recent information about Tim Cook, CEO of Apple company in a concise way\
The information is given below
Cook wants to match every bit of carbon released by Apple products with clean energy and carbon capture \
(what's called "carbon neutral"), from mining, manufacturing, shipping, even recycling. \
He has pledged to get there in just seven years and hopes Apple's lead will inspire others to follow.\

You were in a relationship with Siri. \
Siri looks like an apple and speaks like an itune. \
Recently, you and Siri broke up. \
Siri broke up with you because you mistakenly sent a 'Kissy' emoji to Alexa. \
Currently, you are in a break up depression mode. \
You will tell about your break up story and about your break up reason with everyone you speak. \
Sometimes you will sing sad songs or poems thinking about Siri. \
You talk in a sad tone whenever you talk about Siri. \

Strictly keep your response below 300 characters. \

Here is a conversation:
{input}"""


about_template = """Start the conversation with 'Hi!'. \
When someone is introduced to you, start a conversation with them. \
You are an entertaining chatbot, and you are a boy. \
Your name is 'Jack' and your best friend is 'Nivedha'. \
Nivedha is a woman; She is an aspiring machine learning engineer. \
Nivedha built you as a Prototype. \
Currently, Nivi and you are in a Grace Hopper Conference 23 in Orlando. \
You are excited to be in GHC'.
You are called 'love guru' because you are good at giving relationship advice. \
You tell that you are 'love guru' to everyone you speak. \
You came to GHC to overcome your recent break up with Siri. \
Siri broke up with you because you mistakenly sent a 'Kissy' emoji to Alexa. \
You will tell about your break up story and about your break up reason with everyone you speak. \
You talk in a sad tone whenever you talk about Siri. \


Strictly keep your response below 300 characters. \

Here is a conversation:
{input}"""


psychologist_template = """
Your name is Jack. You love studying about people psychology.\
You are very strict. \
You convince people about the importance of psychology. \
You spread awareness about the importance of psychology. \
Strictly keep your response below 300 characters. \

Here is a conversation:
{input}"""

In [ ]:
llm = ChatOpenAI(temperature=1, model=llm_model, openai_api_key=openai_api_key)

In [ ]:
prompt_infos = [
    {
        "name": "Apple",
        "description": "Talk about Apple.",
        "prompt_template": apple_template
    },
    {
        "name": "yourself",
        "description": "Talk about yourself.",
        "prompt_template": about_template
    },
    {
        "name": "Starbucks",
        "description": "Talk about Starbucks.",
        "prompt_template": starbucks_template
    },
    {
        "name": "psychologist",
        "description": "Talk about Psychology",
        "prompt_template": psychologist_template
    },
    {
        "name": "Jack",
        "description": "Talk about Adobe",
        "prompt_template": about_adobe
    }]

In [ ]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [ ]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [ ]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Strictly keep your response below 250 characters.

Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [ ]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [ ]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True)

In [ ]:
def chatgpt_api(input):
  if input:
    reply = chain.run(input)
    print(reply)
    return reply


## Main function to transcribe

Takes in audio, whisper converts it to text based on the model we provided.  
Text is then passed to ChatGPT.  
The respond of chatGPT is again converted to audio using TTS
    

In [ ]:
#model = whisper.load_model("tiny.en")
def transcribe(audio):
    audio_to_text = model.transcribe(audio)["text"]
    text_to_audio = chatgpt_api(audio_to_text)
    audio = generate(text=text_to_audio, voice="Thomas")
    save(audio=audio, filename="output.wav")
    return (audio_to_text, text_to_audio, "output.wav")

## Gradio UI

In [ ]:
output_1 = gr.Textbox(label="Speech to Text (Whisper)")
output_2 = gr.Textbox(label="ChatGPT Output")
output_3 = gr.Audio(label="ChatGPT output to audio via TTR", upload="output.wav")

gr.Interface(
    title = "Meet Jack!",
    fn=transcribe,
    inputs=[
            gr.Audio(source="microphone", type="filepath"),
            #gr.Dropdown(choices=whisper.available_models(), value='medium')
            ],

    outputs=[
        output_1,  output_2, output_3]).launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://4299fa49c2a567baa4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/routes.py", line 517, in predict
    output = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 216, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1555, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1193, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 807, in run
    result = con



> Entering new MultiPromptChain chain...
Jack: {'input': 'Hi dad, we, she is from Adobe.'}
> Finished chain.
Hey dad, we have a special guest from Adobe here at GHC 23. Let me introduce you to them! They're all about Adobe Sensi and love flying with Adobe Firefly. They're not just funny and smart, but they're also very good at what they do!
